In [49]:
import keras 
from keras.datasets import mnist 
from keras.models import Sequential 
from keras.layers import Convolution2D, Dense, Flatten, MaxPooling2D
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils import to_categorical
from keras.optimizers import Adam

In [97]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1)
x_test= x_test.reshape(10000, 28, 28, 1)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
for i in range(10000):
    a = "image/x_test/" + str(i)
    b = "image/y_test/" + str(i)
    np.savetxt(a, x_test[i,:,:,0])
    np.savetxt(b, y_test[i])

In [98]:
model = Sequential()
model.add(Convolution2D(16, (3, 3), padding="SAME", input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='SAME'))

model.add(Flatten())
model.add(Dense(units=20, activation="relu"))
model.add(Dense(units=10, activation="softmax"))
adam = Adam(lr=0.0001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [99]:
model.fit(x_train, y_train, batch_size=100, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 10s 162us/step - loss: 5.0178 - acc: 0.5843
Epoch 2/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.7838 - acc: 0.8040
Epoch 3/5
60000/60000 [==============================] - 9s 150us/step - loss: 0.5041 - acc: 0.8812
Epoch 4/5
60000/60000 [==============================] - 9s 151us/step - loss: 0.3921 - acc: 0.9045
Epoch 5/5
60000/60000 [==============================] - 9s 151us/step - loss: 0.3140 - acc: 0.9221


In [100]:
print(model.evaluate(x_test, y_test, batch_size=100))

10000/10000 [==============================] - 1s 89us/step
[0.3163654836453497, 0.9234000045061111]


In [101]:
model.save('model/weights.h5') #保存权重

In [102]:
model.weights

[<tf.Variable 'conv2d_16/kernel:0' shape=(3, 3, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv2d_16/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'dense_29/kernel:0' shape=(3136, 20) dtype=float32_ref>,
 <tf.Variable 'dense_29/bias:0' shape=(20,) dtype=float32_ref>,
 <tf.Variable 'dense_30/kernel:0' shape=(20, 10) dtype=float32_ref>,
 <tf.Variable 'dense_30/bias:0' shape=(10,) dtype=float32_ref>]

In [107]:
convCount = 1;
denseCount = 1;
for layer in model.layers:
    weights = layer.get_weights()
    if weights == []:
        continue
    if str(layer).find('Conv2D') != -1:
        layerNum = np.shape(layer.get_weights()[0])[3]
        imageNum = np.shape(layer.get_weights()[0])[2]
        biasname = "layerData/Conv" + str(convCount) + "/bias"
        np.savetxt(biasname, weights[1])
        for j in range(imageNum):
            for i in range(layerNum):
                filename = "layerData/Conv" + str(convCount) + "/imageNum" + str(j + 1) + "/" + str(i)
                np.savetxt(filename, weights[0][:,:,j,i])
        convCount += 1
    elif str(layer).find('Dense') != -1:
        np.savetxt("layerData/Dense" + str(denseCount) + "/weight", weights[0])
        np.savetxt("layerData/Dense" + str(denseCount) + "/bias", weights[1])
        denseCount += 1